In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scipy.io import loadmat
import os
import numpy as np
import pandas as pd
from quad import loadQuad, Quads #custom class for quad data
import tdt
import matplotlib.pyplot as plt

### Load data for given animal/date 
(Diego = Subj. 1, Pacho = Subj. 2)

In [ ]:
animal = 'Pancho'
date = '251119'


In [ ]:
#NOTE: Change paths to local dirs in this function >>
quad = loadQuad(animal, date)

### Check out data structures
- ```quad.prettyBeh``` is the data loaded out of monkey logic

- ```quad.prettyTdt``` is the raw data from our tdt recording system
    - This includes LFP and raw voltage signal but these are not included for sake of file sizes

- ```quad.Dat``` is the merged dataframe, pulls information from Beh and combines it with timing precision in Tdt

- ```quad.SpikeTimes``` is dataframe with spike times loaded from kilosort. These have been automatically clustered by ks and manually curated for 'questionable' spikes.

** All timing relating things are handled automatically to align event codes with kilosort spike times. Note that 'photodiode_time' in Tdt/Dat is the most correct timing to align with kilosort. 

** 'sample_index' is also the most accurate indexing variable, with one 'sample_index' per sample that the monkey actually saw (whether they held fixation or not). Sample index DOES NOT include cases where no sample was shown (i.e. broke fixation during the 200ms fixation period). In the tdt data you will see instances of this where it looks like 'fix_cue' followed immediately by 'sample_off' and 'timeout'. The fact that it says 'sample_off' (even with no 'sample_on') is just an unintended part of the monkeylogic code (i.e. it runs toggleobject(sample,'status','off') whether or not there is a toggleobject(sample,'status,'on)).

In [ ]:
quad.prettyBeh

In [242]:
quad.prettyTdt

,trial_ml2,stim_index,code_type,on,off,photodiode_time
0,1,NaN,trial_start,98.145239,98.174075,98.159657
1,1,NaN,fix_cue,98.174075,98.858476,98.196847
2,1,NaN,sample_off,98.858476,98.891121,98.898737
3,1,NaN,timeout,98.891121,100.108370,98.898737
4,1,NaN,trial_end_blue,100.108370,102.094070,100.116724
...,...,...,...,...,...,...
16451,92,NaN,stim_in_cond_0,5907.448832,5907.597230,5907.488322
16452,92,NaN,sample_off,5907.597230,5907.630653,5907.635778
16453,92,NaN,timeout,5907.630653,5908.847862,5907.635778
16454,92,NaN,trial_end_blue,5908.847862,5910.832456,5908.855731


In [ ]:
quad.Dat

In [ ]:
tdt_stim_inds = quad.prettyTdt['stim_index'].dropna().to_numpy()
beh_stim_inds = quad.prettyBeh['stim_index'].to_numpy()

In [ ]:
quad.spikeTimes

### Plotting!

I have included two plot functions, one to make a PSTH and one to make a raster. Functions have descriptions for use under definition. The main thing to know is what to do with ```params```

```params``` should be a dict formatted with a column name and a list of entries in that column you want to keep. The plotting functions will then filter the data to match those parameters. The PSTH plot also has a 'group_by' argument which you can use to group and average activity over different variables. Examples below.

** Note right now the plotting functions can only align to 'sample_on', the code shouldn't be too hard to write for other trial events, but wasn't sure if you would end up needing that functionality.

** All plotting functions return a dict of figs indexed by 'unit_index'. The titles have the channel number.

In [ ]:
#Plot PSTH
conditions_plot = list(range(1,50))
params = {
    'condition': conditions_plot, #plot only for these conditions
    'fixation_success_binary': [True] #only plot when fixation is successful
    #You can filter by any column/value pair here, as long as the column is present in 'Dat'
}
channel = 187
fig_dict = quad.plotPSTH(channel, params, group_by='fixation_success_binary')

In [ ]:
#Plot raster
channel = 187

params = {
'fixation_success_binary': [True]
# 'stim_index': quad.prettyBeh['stim_index'].to_numpy()[np.r_[0:200, -200:-1]]
}

fig_dict = quad.plotRaster(channel,params, window = (.4,1))


### Additional functions in quad.py

```filter_df``` = function to filter df given dict (structured like params above). Filters df by column/value pairs and returns the filter df. Useful for plotting.

```group_and_average``` = useful function for plotting, will take dat and group by unique values in a particular column and then average over 'spike_counts'. Right now, for example you could make one psth for each unique stimulus. If you wanted to do something like regular vs irregular though, you could edit this function to be more flexible to that.

```getChannelNumOrRegionName``` = A flexible function for getting the channels associated with a region or the region associated with a channel. You can look at the bottom of the file 'MAP_CHANNEL_TO_REGION' is a dict with all this info.